#Read data

In [0]:
expedia_df = spark.read.format("delta").table("silver.expedia_processed")
hotel_weather_df = spark.read.format("delta").table("silver.hotel_weather_processed")

#Decrtypt PII fields

In [0]:
%run "../utils/configuration"

In [0]:
%run "../utils/encryption_helper"

In [0]:
encryption_helper = EncryptionHelper(dbutils)
try:
    expedia_df = encryption_helper.decrypt_dataframe(expedia_df, common_pii_fields.get("expedia"))
    hotel_weather_df = encryption_helper.decrypt_dataframe(hotel_weather_df, common_pii_fields.get("hotel_weather"))    
except Exception as e:
    print(f'Error during decryption of processed data: {e}')
    raise

#Get Top 10 Hotels with Maximum Absolute Temperature Difference by Month

Get _`monthly_temp_diff`_ -> highest temperature difference for every hotel in every month

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW monthly_temp_diff AS
SELECT
  id AS hotel_id,
  MONTH(wthr_date) AS month,
  MAX(avg_tmpr_c) - MIN(avg_tmpr_c) AS temp_diff
FROM silver.hotel_weather_processed
GROUP BY
  id,
  MONTH(wthr_date);
CACHE TABLE monthly_temp_diff;

Get top 10 hotels with highest temp difference for every month

In [0]:
%sql
CREATE OR REPLACE TABLE gold.top_10_temp_diff_monthly 
USING DELTA
AS
SELECT
  hotel_id,
  month,
  ABS(temp_diff) AS temp_diff
FROM (
  SELECT
    hotel_id,
    month,
    temp_diff,
    ROW_NUMBER() OVER (PARTITION BY month ORDER BY ABS(temp_diff) DESC) AS rn
  FROM monthly_temp_diff
)
WHERE rn <= 10
ORDER BY
  month ASC,
  ABS(temp_diff) DESC;

#Get Top 10 Most Visited Hotels per Month 
 Where a "visit" is considered on day spent at hotel, not presense in given month - e.g. visit from 25.04 to 02.05, will be count as 6 visits in April and 2 in May, not 1 visit for both. This should be a more precise approach

Get _`date_series`_ -> singular dates for every day of visit + filtering

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW date_series AS
SELECT
  hotel_id,
  EXPLODE(
    SEQUENCE(
      srch_ci,
      srch_co,
      INTERVAL 1 DAY
    )
  ) AS visit_date
FROM silver.expedia_processed
WHERE srch_ci <= srch_co;
CACHE TABLE date_series;

Get _`monthly_visits`_ -> group visit days by hotel and month and count them

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW monthly_visits AS
SELECT
  hotel_id,
  MONTH(visit_date) AS visit_month,
  COUNT(*) AS monthly_count
FROM date_series
GROUP BY
  hotel_id,
  visit_month;
CACHE TABLE monthly_visits;

Get top 10 hotels for every month

In [0]:
%sql
CREATE OR REPLACE TABLE gold.top_10_busiest_hotels_monthly
USING DELTA
AS
SELECT
  hotel_id,
  visit_month,
  monthly_count
FROM monthly_visits
QUALIFY ROW_NUMBER() OVER (PARTITION BY visit_month ORDER BY monthly_count DESC) <= 10
ORDER BY
  visit_month ASC,
  monthly_count DESC;

# Get Weather Trend for Extended Stays (more than 7 days)

Enable range join optimization globally for the session

In [0]:
%sql
SET spark.databricks.optimizer.rangeJoin.enabled = true;

Get _`stays`_ -> filter stays longer than 7 days and calculate stay length

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW stays AS
SELECT
    e.user_id,
    e.hotel_id,
    CAST(e.srch_ci AS DATE) AS checkin_date,
    CAST(e.srch_co AS DATE) AS checkout_date,
    DATEDIFF(CAST(e.srch_co AS DATE), CAST(e.srch_ci AS DATE)) AS stay_length
FROM silver.expedia_processed e
WHERE srch_ci IS NOT NULL
  AND srch_co IS NOT NULL
  AND DATEDIFF(CAST(e.srch_co AS DATE), CAST(e.srch_ci AS DATE)) > 7;
CACHE TABLE stays;

Get _`weather_per_day`_ -> join stays with hotel weather data for each day of the stay (range join optimized)

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW weather_per_day AS
SELECT
    s.user_id,
    s.hotel_id,
    hw.wthr_date,
    hw.avg_tmpr_c
FROM stays s
JOIN silver.hotel_weather_processed hw
  ON s.hotel_id = hw.id
 AND hw.wthr_date >= s.checkin_date
 AND hw.wthr_date <= s.checkout_date;
CACHE TABLE weather_per_day;

Get _`weather_ranked`_ -> retrieve first and last day temperatures for each stay

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW weather_ranked AS
SELECT
    user_id,
    hotel_id,
    wthr_date,
    avg_tmpr_c,
    FIRST_VALUE(avg_tmpr_c) OVER (PARTITION BY user_id, hotel_id ORDER BY wthr_date ASC) AS first_temp,
    LAST_VALUE(avg_tmpr_c) OVER (
        PARTITION BY user_id, hotel_id
        ORDER BY wthr_date ASC
        ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
    ) AS last_temp
FROM weather_per_day;
CACHE TABLE weather_ranked;

Get _`weather_trend_final`_ -> calculate temperature trend and average temperature during stay

In [0]:
%sql
CREATE OR REPLACE TABLE gold.weather_trend_extended_stay
USING DELTA
AS
SELECT
    user_id,
    hotel_id,
    last_temp - first_temp AS temp_trend,
    AVG(avg_tmpr_c) AS avg_temp_stay
FROM weather_ranked
GROUP BY user_id, hotel_id, first_temp, last_temp

In [0]:
%sql
UNCACHE TABLE stays;
UNCACHE TABLE weather_per_day;
UNCACHE TABLE weather_ranked;
UNCACHE TABLE date_series;
UNCACHE TABLE monthly_temp_diff;